In [1]:
import pandas as pd

### Évolution des prix dans le temps par genre ? 

In [2]:
games=pd.read_csv("final/games.csv")
games.head()

,Unnamed: 0,gameid,title,platform,developers,publishers,genres,supported_languages,release_date,plateforme,main_genre,supported_languages_count,release_year
0,0,749375_playstation,Medieval Match Master,PS4,['Erik Games'],['Erik Games'],['Puzzle'],['unknown'],2024-11-22,playstation,Puzzle,131888,2024
1,1,749839_playstation,Sunforge Solar Survival,PS4,['GrizzlyGames'],['GrizzlyGames'],unknown,['unknown'],2024-11-21,playstation,unknown,131888,2024
2,2,749347_playstation,Find Love Or Die Trying,PS5,['Sunseeker Games'],['Ratalaika Games'],['Visual Novel'],['unknown'],2024-11-22,playstation,Visual Novel,131888,2024
3,3,749341_playstation,Medieval Match Master,PS4,['Erik Games'],['Erik Games'],['Puzzle'],['unknown'],2024-11-22,playstation,Puzzle,131888,2024
4,4,749097_playstation,Bust A Block 2,PS5,['ThiGames'],['ThiGames'],['Puzzle'],['unknown'],2024-11-21,playstation,Puzzle,131888,2024


In [3]:
prices=pd.read_csv("final/prices.csv")
prices.head()

,Unnamed: 0,gameid,usd,eur,gbp,jpy,rub,date_acquired,plateforme,acquired_year
0,0,749347_playstation,10.39,10.39,7.99,1408.0,929.0,2025-02-22,playstation,2025
1,1,749281_playstation,1.49,1.49,1.19,220.0,NaN,2025-02-22,playstation,2025
2,2,749839_playstation,14.99,14.99,11.99,2310.0,1069.0,2025-02-22,playstation,2025
3,3,749280_playstation,1.49,1.49,1.19,220.0,NaN,2025-02-22,playstation,2025
4,4,749375_playstation,0.99,0.49,0.39,165.0,71.0,2025-02-22,playstation,2025


In [15]:
prices.shape

(4499727, 10)

In [32]:
# merger deux tables 

games_prices=games.merge(
  prices,
  left_on =['gameid'],
  right_on =['gameid'],
  how='left'    
)

In [33]:
games_prices.head()

,Unnamed: 0_x,gameid,title,platform,developers,publishers,genres,supported_languages,release_date,plateforme_x,...,release_year,Unnamed: 0_y,usd,eur,gbp,jpy,rub,date_acquired,plateforme_y,acquired_year
0,0,749375_playstation,Medieval Match Master,PS4,['Erik Games'],['Erik Games'],['Puzzle'],['unknown'],2024-11-22,playstation,...,2024,4,0.99,0.49,0.39,165.0,71.0,2025-02-22,playstation,2025
1,0,749375_playstation,Medieval Match Master,PS4,['Erik Games'],['Erik Games'],['Puzzle'],['unknown'],2024-11-22,playstation,...,2024,30763,0.99,0.49,0.39,165.0,71.0,2025-02-24,playstation,2025
2,1,749839_playstation,Sunforge Solar Survival,PS4,['GrizzlyGames'],['GrizzlyGames'],unknown,['unknown'],2024-11-21,playstation,...,2024,2,14.99,14.99,11.99,2310.0,1069.0,2025-02-22,playstation,2025
3,1,749839_playstation,Sunforge Solar Survival,PS4,['GrizzlyGames'],['GrizzlyGames'],unknown,['unknown'],2024-11-21,playstation,...,2024,30765,14.99,14.99,11.99,2310.0,1069.0,2025-02-24,playstation,2025
4,2,749347_playstation,Find Love Or Die Trying,PS5,['Sunseeker Games'],['Ratalaika Games'],['Visual Novel'],['unknown'],2024-11-22,playstation,...,2024,0,10.39,10.39,7.99,1408.0,929.0,2025-02-22,playstation,2025


In [8]:
games_prices.shape

(4473113, 5)

In [34]:
# remplacer les valeurs manquantes du dollars par une valuer connue d'une autre devise en appliqaunt le taux de conversion 

# Taux de conversion vers USD
eur_to_usd = 1.08
gbp_to_usd = 1.26
jpy_to_usd = 0.0066
rub_to_usd = 0.011

games_prices['usd'] = games_prices['usd'].where(games_prices['usd'].notna(), games_prices['jpy'].where(games_prices['jpy']. notna())*jpy_to_usd)
games_prices['usd'] = games_prices['usd'].where(games_prices['usd'].notna(), games_prices['eur'].where(games_prices['eur']. notna())*eur_to_usd)
games_prices['usd'] = games_prices['usd'].where(games_prices['usd'].notna(), games_prices['gbp'].where(games_prices['gbp']. notna())*gbp_to_usd)
games_prices['usd'] = games_prices['usd'].where(games_prices['usd'].notna(), games_prices['rub'].where(games_prices['rub']. notna())*rub_to_usd)


In [38]:
games_prices.isnull().sum()

Unnamed: 0_x                       0
gameid                             0
title                              0
platform                           0
developers                         0
publishers                         0
genres                             0
supported_languages                0
release_date                       0
plateforme_x                       0
main_genre                         0
supported_languages_count          0
release_year                       0
Unnamed: 0_y                       0
usd                           911289
eur                          1711185
gbp                           915495
jpy                           963799
rub                          1046103
date_acquired                      0
plateforme_y                       0
acquired_year                      0
dtype: int64

In [39]:
# creer une table avec les colonnes dont on a besoin pour répondre a la quesiton -historique de l'évolution

games_prices=games_prices[['gameid', 'main_genre', 'usd','date_acquired','release_date']]

games_prices.head()

,gameid,main_genre,usd,date_acquired,release_date
0,749375_playstation,Puzzle,0.99,2025-02-22,2024-11-22
1,749375_playstation,Puzzle,0.99,2025-02-24,2024-11-22
2,749839_playstation,unknown,14.99,2025-02-22,2024-11-21
3,749839_playstation,unknown,14.99,2025-02-24,2024-11-21
4,749347_playstation,Visual Novel,10.39,2025-02-22,2024-11-22


In [40]:
games_prices.isnull().sum()

gameid                0
main_genre            0
usd              911289
date_acquired         0
release_date          0
dtype: int64

In [71]:
# enlever les lignes où on n'a aucun tarif
games_prices=games_prices.dropna()

In [72]:
games_prices.isnull().sum()

gameid           0
main_genre       0
usd              0
date_acquired    0
release_date     0
year             0
dtype: int64

In [73]:
# converir release_date en datetime et extraire l'année

games_prices['year'] = pd.to_datetime(games_prices['release_date']).dt.year

In [74]:
games_prices.head()

,gameid,main_genre,usd,date_acquired,release_date,year
0,749375_playstation,Puzzle,0.99,2025-02-22,2024-11-22,2024
1,749375_playstation,Puzzle,0.99,2025-02-24,2024-11-22,2024
2,749839_playstation,unknown,14.99,2025-02-22,2024-11-21,2024
3,749839_playstation,unknown,14.99,2025-02-24,2024-11-21,2024
4,749347_playstation,Visual Novel,10.39,2025-02-22,2024-11-22,2024


In [75]:
# Calculer la moyenne des prix par année et par genre 

avg_price_by_year_genre = (games_prices
    .groupby(['year', 'main_genre']) 
    [['usd']]
    .mean()                        
    .reset_index()                 
    .round(2)                      
)

avg_price_by_year_genre

,year,main_genre,usd
0,1997,Action,7.49
1,1998,Action,8.03
2,1999,Action,4.07
3,2000,Action,6.03
4,2001,Action,6.47
...,...,...,...
1001,2025,Puzzle,3.42
1002,2025,Simulation,78.97
1003,2025,Strategy,6.35
1004,2025,shooter,24.99


In [101]:
avg_price_by_year_genre.to_csv("powerbi/prix_dans_temps_par_genre_2.csv",index=False)

### Analyse complémentaire pour avoir l'évolution du tarif du genre entre l'apparition et 2025

In [76]:
### taux moyen de différence de prix par genre 
# d'abord la difference d'une annee a l'autre 

avg_price_by_year_genre['price_diff_usd'] = avg_price_by_year_genre.groupby('main_genre')['usd'].diff()
avg_price_by_year_genre['prev_year'] = avg_price_by_year_genre.groupby('main_genre')['year'].shift(1)

result = avg_price_by_year_genre[['year', 'prev_year', 'main_genre', 'usd', 'price_diff_usd']]
# result.dropna(subset=['price_diff_usd']).head()

In [77]:
result

,year,prev_year,main_genre,usd,price_diff_usd
0,1997,NaN,Action,7.49,NaN
1,1998,1997.0,Action,8.03,0.54
2,1999,1998.0,Action,4.07,-3.96
3,2000,1999.0,Action,6.03,1.96
4,2001,2000.0,Action,6.47,0.44
...,...,...,...,...,...
1001,2025,2024.0,Puzzle,3.42,-2.45
1002,2025,2024.0,Simulation,78.97,64.78
1003,2025,2024.0,Strategy,6.35,-4.91
1004,2025,2024.0,shooter,24.99,13.67


In [78]:
result.isnull().sum()

year               0
prev_year         98
main_genre         0
usd                0
price_diff_usd    98
dtype: int64

In [79]:
# ensuite la somme de toutes les differences 


evolution_avg_price_by_genre=avg_price_by_year_genre.groupby('main_genre')[['price_diff_usd']].sum().reset_index()

In [80]:
evolution_avg_price_by_genre

,main_genre,price_diff_usd
0,"""Beat em up""",9.72
1,"""Shoot em up""",-5.00
2,ARCADE,0.60
3,Accounting,-2.36
4,Action,-2.34
...,...,...
93,social,-5.00
94,sports,17.14
95,strategy,1.00
96,unknown,5.53


In [81]:
# trier par ce qui ont plus augmenté et ce qui ont plus baissé 
evolution_avg_price_by_genre.sort_values('price_diff_usd')

,main_genre,price_diff_usd
56,Photo Editing,-46.96
10,Animation & Modeling,-36.48
43,Management,-27.18
23,Design & Illustration,-22.53
12,Audio Production,-21.42
...,...,...
29,Fighting,20.59
20,Collectable Card Game,25.00
70,Simulation Racing,32.10
34,Game Development,43.73


In [82]:
# s'assurer  encore une fois qu'il n'y a pas de valeurs manquantes 
evolution_avg_price_by_genre.isnull().sum()

main_genre        0
price_diff_usd    0
dtype: int64

In [83]:
# garder le prix avec la date la plus ancienne pour chaque genre
# pour cela filtrer sur toutes les lignes qui n'ont pas de valeur pour prev_year 


# Étape 1 : Trouver l'index de l'année la plus ancienne par genre
idx_oldest = result['prev_year'].isna()

# Étape 2 : Filtrer le DataFrame pour ne garder que ces lignes
df_oldest = result.loc[idx_oldest]

df_oldest.head()

,year,prev_year,main_genre,usd,price_diff_usd
0,1997,NaN,Action,7.49,NaN
5,2001,NaN,Strategy,15.99,NaN
8,2003,NaN,Role Playing,14.99,NaN
12,2005,NaN,Board Games,4.99,NaN
13,2005,NaN,Classics,4.99,NaN


In [84]:
# garder que les colonnes utiles ( les deux autes sotn nan)

df_oldest = df_oldest[['year', 'main_genre', 'usd']]


df_oldest.head()

,year,main_genre,usd
0,1997,Action,7.49
5,2001,Strategy,15.99
8,2003,Role Playing,14.99
12,2005,Board Games,4.99
13,2005,Classics,4.99


In [85]:
df_oldest.isnull().sum()

year          0
main_genre    0
usd           0
dtype: int64

In [56]:
df_oldest.shape

(101, 3)

In [86]:
# merger evolution_avg_price_by_genre et df_oldest sur main_genre pour avoir le prix le plus ancien  et 
# et la difference par rapport a la date la plus recente 

evolution_avg_price_by_genre=evolution_avg_price_by_genre.merge(
  df_oldest,
  left_on =['main_genre'],
  right_on =['main_genre'],
  how='left'
)

evolution_avg_price_by_genre.head()

,main_genre,price_diff_usd,year,usd
0,"""Beat em up""",9.72,2010,8.74
1,"""Shoot em up""",-5.00,2007,9.99
2,ARCADE,0.60,2014,9.39
3,Accounting,-2.36,2016,14.99
4,Action,-2.34,1997,7.49


In [87]:
evolution_avg_price_by_genre.shape

(98, 4)

In [ ]:
# calculer pourcentage d'évolution du prix (le taux)

evolution_avg_price_by_genre['rates_price_evolution']=evolution_avg_price_by_genre['price_diff_usd']/evolution_avg_price_by_genre['usd']*100

evolution_avg_price_by_genre


,main_genre,price_diff_usd,year,usd,rates_price_evolution
0,"""Beat em up""",9.72,2010,8.74,111.212815
1,"""Shoot em up""",-5.00,2007,9.99,-50.050050
2,ARCADE,0.60,2014,9.39,6.389776
3,Accounting,-2.36,2016,14.99,-15.743829
4,Action,-2.34,1997,7.49,-31.241656
...,...,...,...,...,...
93,social,-5.00,2016,14.99,-33.355570
94,sports,17.14,2005,9.99,171.571572
95,strategy,1.00,2005,9.99,10.010010
96,unknown,5.53,2008,8.35,66.227545


In [89]:
# trier descending
evolution_avg_price_by_genre.sort_values('rates_price_evolution', ascending=False)

,main_genre,price_diff_usd,year,usd,rates_price_evolution
69,Simulation,71.10,2008,7.87,903.430750
36,Gore,12.77,2013,2.22,575.225225
34,Game Development,43.73,2019,7.99,547.309136
20,Collectable Card Game,25.00,2012,4.99,501.002004
70,Simulation Racing,32.10,2012,9.99,321.321321
...,...,...,...,...,...
26,Education,-18.14,2014,21.86,-82.982617
33,Free to Play,-1.66,2019,1.99,-83.417085
23,Design & Illustration,-22.53,2013,24.49,-91.996733
56,Photo Editing,-46.96,2012,49.99,-93.938788


In [90]:
# renommer les colonnes 

evolution_avg_price_by_genre.columns=['main_genre', 'price_diff_usd', 'genre_first_year', 'genre_first_usd_price_avg', 'price_evolution_rate']
evolution_avg_price_by_genre


,main_genre,price_diff_usd,genre_first_year,genre_first_usd_price_avg,price_evolution_rate
0,"""Beat em up""",9.72,2010,8.74,111.212815
1,"""Shoot em up""",-5.00,2007,9.99,-50.050050
2,ARCADE,0.60,2014,9.39,6.389776
3,Accounting,-2.36,2016,14.99,-15.743829
4,Action,-2.34,1997,7.49,-31.241656
...,...,...,...,...,...
93,social,-5.00,2016,14.99,-33.355570
94,sports,17.14,2005,9.99,171.571572
95,strategy,1.00,2005,9.99,10.010010
96,unknown,5.53,2008,8.35,66.227545


In [91]:
# ajouter une colonne pour avoir un indicatuer du prix en augmentation ou  en baisse 

evolution_avg_price_by_genre['price_rising']=evolution_avg_price_by_genre['price_evolution_rate']>=0

evolution_avg_price_by_genre

,main_genre,price_diff_usd,genre_first_year,genre_first_usd_price_avg,price_evolution_rate,price_rising
0,"""Beat em up""",9.72,2010,8.74,111.212815,True
1,"""Shoot em up""",-5.00,2007,9.99,-50.050050,False
2,ARCADE,0.60,2014,9.39,6.389776,True
3,Accounting,-2.36,2016,14.99,-15.743829,False
4,Action,-2.34,1997,7.49,-31.241656,False
...,...,...,...,...,...,...
93,social,-5.00,2016,14.99,-33.355570,False
94,sports,17.14,2005,9.99,171.571572,True
95,strategy,1.00,2005,9.99,10.010010,True
96,unknown,5.53,2008,8.35,66.227545,True


In [94]:
evolution_avg_price_by_genre=evolution_avg_price_by_genre.sort_values('price_evolution_rate', ascending=False)

evolution_avg_price_by_genre

,main_genre,price_diff_usd,genre_first_year,genre_first_usd_price_avg,price_evolution_rate,price_rising
69,Simulation,71.10,2008,7.87,903.430750,True
36,Gore,12.77,2013,2.22,575.225225,True
34,Game Development,43.73,2019,7.99,547.309136,True
20,Collectable Card Game,25.00,2012,4.99,501.002004,True
70,Simulation Racing,32.10,2012,9.99,321.321321,True
...,...,...,...,...,...,...
26,Education,-18.14,2014,21.86,-82.982617,False
33,Free to Play,-1.66,2019,1.99,-83.417085,False
23,Design & Illustration,-22.53,2013,24.49,-91.996733,False
56,Photo Editing,-46.96,2012,49.99,-93.938788,False


In [102]:
evolution_avg_price_by_genre.to_csv("powerbi/price_evolution_rate_3.csv")

In [99]:
# calculer le pourcentage de genre qui en prix monte ou baisse ( 98 rows * 100)
evolution_avg_price_by_genre.groupby('price_rising').count()/0.98

,main_genre,price_diff_usd,genre_first_year,genre_first_usd_price_avg,price_evolution_rate
price_rising,,,,,
False,42.857143,42.857143,42.857143,42.857143,42.857143
True,57.142857,57.142857,57.142857,57.142857,57.142857
